# Τελικό εργαλείο

In [1]:
import numpy as np
import spacy
from greek_stemmer import GreekStemmer
stemmer = GreekStemmer()
from joblib import dump, load
import nltk
import re
nltk.download('stopwords') 
from nltk.corpus import stopwords
import string
import el_core_news_md
nlp = el_core_news_md.load()
total_stop_words = (nlp.Defaults.stop_words).union(set(stopwords.words('greek')))
total_stop_words = total_stop_words.union({'ή', 'μόνον'})
total_puncts = string.punctuation + '«»–“”\xa0‘•…●\uf0b7◗♥.'
total_stop_words = list(total_stop_words) + list(string.punctuation)
#total_stop_words += [stemming(word) for word in total_stop_words]
stop_words = set(total_stop_words)
tokenized_stop_words = nltk.word_tokenize(' '.join(total_stop_words))

systems_dict = {0: 'κανένα', 1: 'νευρικό',  2: 'κυκλοφορικό', 3: 'λεμφικό-ανοσοποιητικό',  4: 'ουροποιητικό', 5: 'πεπτικό', 
                6: 'αναπνευστικό', 7: 'ερειστικό', 8: 'μυϊκό', 9: 'καλυπτήριο', 10: 'αναπαραγωγικό', 11: 'αισθητήριων οργάνων', 
                12: 'ενδοκρινών αδένων'}

class Tokenizer(object):
    def __init__(self):
        nltk.download('punkt', quiet=True, raise_on_error=True)
        self.stemmer = GreekStemmer()
        
    def _stem(self, token):
        if (token in stop_words):
            return token
        return self.stemmer.stem((removing_accents(token)).upper())
        
    def __call__(self, line):
        tokens = nltk.word_tokenize(line)
        tokens = filter_punctuation(list(tokens))
        tokens = group_numbers(tokens)
        tokens = (self._stem(token) for token in tokens)
        return tokens

def filter_punctuation(words):
    filtered_words = []
    for word in words:
        pun = []
        for letter in word:
            pun.append(letter in total_puncts)

        if not all(pun):
            word = word.strip(total_puncts)
            
            if (len(word)>3):
                for punct in total_puncts:
                    word = word.replace(punct, ' ')
            else:
                for punct in total_puncts:
                    word = word.replace(punct, '')
                    
            tokens = nltk.word_tokenize(word)
            
            for token in tokens:
                filtered_words.append(token)
                
    return filtered_words  

def herbs_from_txt(filename, encoding):
    """
    Reads the content of a txt file and produces a list of strings 
    -in our case a list of search terms- separated by comma.

    Parameters
    ----------
    filename : str
        the path (relative or absolute) of the txt file 
        which contains search terms
    encoding : str
        the encoding of the txt file which contains search terms

    Returns
    -------
    list
        a list of strings - search terms (mutliword or singleword)
    """
    with open(filename,'r',encoding=encoding) as f:
        content = f.readlines()
    content = [x.strip() for x in content] 
    for term in content:
        herbs = term.split(',')
    return herbs

def stemmed_herbs(herbs):
    stemmed_herbs = []
    for herb in herbs:
        stemmed_herbs.append(stemming(herb))
    return stemmed_herbs

def group_numbers(words):
    new_words = []
    for word in words:
        if re.fullmatch(r'[0-9]+', word) != None :
            if int(word) >= 1800 and int(word) <= 2020:
                word = '2000'
            else:
                word = '1'
        new_words.append(word)
    return new_words

    
def ner_predictions(text):
    ner_model = spacy.load('./ner_model/')
    pred_value = ner_model(text)
    
    ent_type = []
    ent_text = []
    for ent in pred_value.ents:
        ent_type.append(ent.label_)
        ent_text.append(ent.text)
    return ent_type, ent_text

def removing_accents(word):
    """
    Removes accents from a given word.
    
    Parameters
    ----------
    word : str
        the word from which we want to remove the accents

    Returns
    -------
    str
        the word without accents
    """
    return (word.replace('ά', 'α').replace('έ', 'ε').replace('ή', 'η').replace('ί', 'ι').replace('ό', 'ο').replace('ύ', 'υ')
            .replace('ώ', 'ω').replace('ϊ', 'ι').replace('ϋ', 'υ').replace('ΐ', 'ι').replace('ΰ', 'υ'))

def stemming(word):
    """
    Implements stemming for a given word.
    
    Parameters
    ----------
    word : str
        the word that is about to be stemmed

    Returns
    -------
    str
        the stem of the given word at uppercase letters
    """
    return stemmer.stem((removing_accents(word)).upper())

def group_same_stems(word_list):
    stems = []
    for word in word_list:
        stem = stemming(word)
        stems.append(stem)
    
    return list(np.unique(stems))
    
def sentence_tokenization(text):
    documents = nltk.sent_tokenize(text)
    final_documents = []
    for j in range(len(documents)):   
        if re.search(r'[.!;][Α-ΩA-Z"«“]', documents[j]) != None :
            sents = re.split(r'([.!;][Α-ΩA-Z"«“])', documents[j])
            sentences = []
            for i in range(len(sents)):
                if i == 0:
                    sentences.append(sents[i])
                elif i%2 == 0:
                     sentences.append(sents[i-1][1] + sents[i])
            for sentence in sentences:
                if len(sentence)>3:
                    final_documents.append(sentence)
        else:
            if len(documents[j])>3:
                final_documents.append(documents[j])
        
    return final_documents
    
    
def main(text):
    plants = []
    organ_systems = []
    ent_type, ent_text = ner_predictions(text)
    response = 'Το δοσμένο κείμενο '
    if ent_type.count('PLANT') == 0 :
        response += 'δεν '
    else:
        for i in range(len(ent_type)):
            if ent_type[i] == 'PLANT':
                plants.append(ent_text[i])
                
        grouped_plants = []
        known_plants = herbs_from_txt('../data/plants.txt', 'utf-8')
        stemmed_known_plants = stemmed_herbs(known_plants)
        for plant in plants:
            stemmed_plant = stemming(plant)
            if stemmed_plant in stemmed_known_plants:
                pos = stemmed_known_plants.index(stemmed_plant)
                grouped_plants.append(known_plants[pos])
            else:
                grouped_plants.append(plant)
            
        grouped_plants = list(np.unique(grouped_plants))    
    
        clf = load('svc.joblib')
        tf_idf = load('tf_idf_vectorizer.joblib')
        sentences = sentence_tokenization(text)
        tf_idf_sentences = tf_idf.transform(sentences)
        preds = clf.predict(tf_idf_sentences)
        systems = [systems_dict.get(pred) for pred in preds if pred!=0]
        organ_systems = list(np.unique(systems))
        if not organ_systems:
            response += 'δεν '
            
    response += 'είναι σχετικό με την εθνοφαρμακολογία.'
    print(response)
    if plants:
        print('\n')
        print('Αναφερόμενα φυτά και βότανα:')
        print(*grouped_plants, sep=', ')
    if organ_systems:
        print('\n')
        print('Πιθανώς επηρεαζόμενα συστήματα οργάνων:')
        print(*organ_systems, sep=', ')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
main('Κάνοντας μερικές μικρές αλλαγές στην καθημερινότητά σου, όπως να τρέφεσαι καλύτερα ή να κινείσαι λίγο περισσότερο μέσα στην ημέρα, μπορείς να βελτιώσεις σημαντικά την υγεία της καρδιάς σου και να αποφύγεις σοβαρές επιπλοκές όπως το έμφραγμα και το εγκεφαλικό. Ένα πρώτο και πολύ απλό βήμα για να πετύχεις αυτόν τον στόχο είναι να βάλεις στη ζωή σου τα παρακάτω βότανα και μπαχαρικά! Κανέλα Δίνει υπέροχο άρωμα στο φαγητό, στα σπιτικά γλυκά αλλά και στα ροφήματα και επίσης βοηθά στον έλεγχο του σακχάρου. Τα υψηλά επίπεδα σακχάρου οδηγούν μακροπρόθεσμα στην εκδήλωση διαβήτη και η πάθηση αυτή είναι ένας από τους βασικότερους παράγοντες κινδύνου για το έμφραγμα και το εγκεφαλικό. Σκόρδο Το σκόρδο έχει ισχυρή αντιφλεγμονώδη και αντιμικροβιακή δράση. Οι καρδιολόγοι το συνιστούν ανεπιφύλακτα κυρίως για τα αντιοξειδωτικά που παρέχει αλλά και γιατί αραιώνει το αίμα, επομένως μειώνει τον κίνδυνο θρόμβωσης. Τζίντζερ Το καυτερό τζίντζερ χρησιμοποιείται παραδοσιακά ως «γιατροσόφι» για τη δυσπεψία, είναι όμως ωφέλιμο και για την καρδιά. Αραιώνει το αίμα όπως και το σκόρδο και επίσης μπορεί να συμβάλει στη μείωση της χοληστερόλης. Κουρκουμάς Ο κουρκουμάς περιέχει ένα ισχυρό συστατικό, την κουρκουμίνη, η οποία δρα ενάντια στις φλεγμονές και έχει αντιοξειδωτική δράση, μειώνοντας έτσι τον κίνδυνο καρκίνου αλλά και καρδιοπάθειας. Δεντρολίβανο Ενισχύει την παραγωγή ερυθρών αιμοσφαιρίων και διευκολύνει την κυκλοφορία του αίματος. Οι διεργασίες αυτές βοηθούν ώστε να παρέχεται περισσότερο οξυγόνο στα ζωτικά όργανα και τα επιμέρους συστήματα του οργανισμού. Δες κι αυτό: Παγκόσμια Ημέρα Καρδιάς: Δες 3 απλούς τρόπους να προστατεύσεις τη δική σου! Το top 5 των καλύτερων τροφών για την υγεία της καρδιάς Υπερβολικό βάρος vs αποχή από την άσκηση: Τι είναι πιο επιβαρυντικό για την καρδιά;')

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['Δ', 'ἈΛΛ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Το δοσμένο κείμενο είναι σχετικό με την εθνοφαρμακολογία.


Αναφερόμενα φυτά και βότανα:
δεντρολίβανο, κανέλα, κουρκουμάς, σκόρδο, τζίντζερ


Πιθανώς επηρεαζόμενα συστήματα οργάνων:
κυκλοφορικό, νευρικό, ουροποιητικό, πεπτικό


In [3]:
main('Ο Σάκης (Αναστάσιος) Ρουβάς (Μαντούκι Κέρκυρας, 5 Ιανουαρίου 1972) είναι Έλληνας τραγουδιστής, συνθέτης, επιχειρηματίας, και πρώην αθλητής του άλματος επί κοντώ. Από μικρή ηλικία ασχολήθηκε με τον στίβο, κατακτώντας αρκετά μετάλλια ως μέλος της Ελληνικής Εθνικής Ομάδας τη δεκαετία του 1980. Η μουσική του σταδιοδρομία ξεκίνησε το 1991, σε ηλικία 19 ετών, ως pop ερμηνευτής ενώ συνδύαζε τραγούδι, χορογραφία και σύνθεση του κομματιού. Ο Ρουβάς έχει κερδίσει 6 βραβεία στα Arion Music Awards, 15 στα Pop Corn Music Awards και 22 στα Mad Video Music Awards. Επίσης, έχει ανακηρυχθεί "Άντρας της χρονιάς", έχει κερδίσει ένα βραβείο στα MTV Europe Music Awards και ένα στα World Music Awards. Το 2009, το περιοδικό "Down Town", του έδωσε τον τίτλο "Καλύτερος διασκεδαστής της δεκαετίας", ενώ το "Forbes" τον κατέταξε στην τρίτη θέση με τους πιο σημαντικούς καλλιτέχνες στην Ελλάδα.')

Το δοσμένο κείμενο δεν είναι σχετικό με την εθνοφαρμακολογία.


In [57]:
main('Ορισμένα θρεπτικά στοιχεία βελτιώνουν τις παθήσεις που συνδέονται με την αναπνευστική οδό, κάτι που αποδεικνύεται χρήσιμο αν παρουσιάζετε άσθμα, αλλεργίες ή προσπαθείτε να κόψετε το κάπνισμα. 6 τροφές που συμβάλλουν στην υγιή λειτουργία των πνευμόνων Μήλα Τα μήλα είναι εφοδιασμένα με φαινολικά οξέα και φλαβονοειδή που είναι γνωστές για τη μείωση των φλεγμονών στην αναπνευστική οδό, κοινό στοιχείο τόσο για το άσθμα όσο και για τη δύσπνοια. Ελαιόλαδο Τα μονοακόρεστα και πολυακόρεστα λιπαρά και τα φυτοθρεπτικά στοιχεία που βρίσκουμε στο ελαιόλαδο είναι ιδανικά όχι μόνο για την επιδερμίδα, τα μαλλιά και την καρδιά• παίζουν επίσης ρόλο και στην υγεία των πνευμόνων. Για την ακρίβεια, το ελαιόλαδο βοηθά στην καταπολέμηση κινδύνων που σχετίζονται με την ατμοσφαιρική ρύπανση, όπως αυξημένη αρτηριακή πίεση και προβλήματα στα αιμοφόρα αγγεία – παράγοντες που μπορούν να μειώσουν την παροχή οξυγόνου, να κάνουν την καρδιά μας να χτυπά ταχύτερα και την αναπνοή μας δυσκολότερη. Καφές Μία κούπα καφέ κάνει περισσότερα από το να «ξυπνά» τον εγκέφαλό μας – θα μπορούσε επίσης να βοηθά στην ανακούφιση συμπτωμάτων του άσθματος. Η καφεΐνη είναι ένα ήπιο βρογχοδιασταλτικό, όμως δεν συγκρίνεται με τα φάρμακα. Σολομός Τα ωμέγα-3 λιπαρά οξέα που βρίσκουμε σε ψάρια όπως ο σολομός βοηθούν στη μείωση των φλεγμονών στους πνεύμονες και επίσης καταπολεμούν τα βακτήρια. Πράσινο τσάι Μια ζεστή κούπα πράσινου τσαγιού είναι εφοδιασμένα με αντιοξειδωτικά που ηρεμούν το σώμα, μειώνουν τη φλεγμονή και προάγουν την επούλωση. Αλλά ένα σημαντικό επίσης συστατικό είναι η κουερσετίνη, ένα αντιοξειδωτικό που δρα ως φυσικό αντιισταμινικό. Αυτό σημαίνει ότι επιβραδύνει την απελευθέρωση της ισταμίνης και άλλων φλεγμονωδών χημικών στο σώμα που προκαλούν τα συμπτώματα αλλεργίας. Πορτοκάλι Φρούτα, όπως η γλυκιά κολοκύθα, τα πορτοκάλια και η παπάγια είναι γεμάτα αντιοξειδωτικά, φιλικά προς τους πνεύμονες, με πιο σημαντική τη βιταμίνη C. Η βιταμίνη C είναι γνωστή για την καταπολέμηση μολύνσεων και φλεγμονών, και μια έκθεση μελετών που δημοσιεύτηκε στο Allergy, Asthma & Clinical Immunology απέδειξε ότι μπορεί να παίζει έναν ρόλο στη μείωση της συχνότητας εμφάνισης άσθματος που προκαλείται από την άσκηση κατά 52%. Πηγές: https://aacijournal.biomedcentral.com/articles/10.1186/1710-1492-10-58')

Το δοσμένο κείμενο είναι σχετικό με την εθνοφαρμακολογία.


Αναφερόμενα φυτά και βότανα:
Πράσινο τσάι, κολοκυθιά, μηλιά, πορτοκαλιά


Πιθανώς επηρεαζόμενα συστήματα οργάνων:
αναπνευστικό, καλυπτήριο, κυκλοφορικό, λεμφικό ανοσοποιητικό, νευρικό, πεπτικό


In [58]:
main('Το νόμπελ μοιράστηκαν οι επιστήμονες Roger Penrose, Reinhard Genzel και Andrea Ghez.   «Μας πήρε λίγα λεπτά παραπάνω να επικοινωνήσουμε με τους νικητές, για αυτό καθυστερήσαμε», απολογήθηκε ο γενικός γραμματέας του ιδρύματος, Göran K. Hansson μπαίνοντας στην αίθουσα. «Το φετινό βραβείο, αφορά τα πιο βαθιά μυστικά του σύμπαντος», συμπλήρωσε, προαναγγέλλοντας τους επιστήμονες που τιμήθηκαν με το Νόμπελ.  Advertisement: 0:04   Στον Βρετανό Roger Penrose απονεμήθηκε το μισό νόμπελ «για την ανακάλυψη πως οι μαύρες τρύπες προβλέπονται από τη Γενική Θεωρία της Σχετικότητας». Το άλλο μισό νόμπελ, μοιράστηκαν ο Γερμανός Reinhard Genzel και η Αμερικανίδα Andrea Ghez «για τη ανακάλυψη ενός αντικειμένου τεράστιας μάζας στο κέντρο του Γαλαξία μας».    Η Andrea Ghez έγινε η τέταρτη γυναίκα στην ιστορία των βραβείων που λαμβάνει νόμπελ Φυσικής.  Πηγή: www.lifo.gr')

Το δοσμένο κείμενο δεν είναι σχετικό με την εθνοφαρμακολογία.
